In [ ]:
# !pip install yfinance pandas numpy matplotlib ipywidgets

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from typing import NoReturn

def display_title() -> NoReturn:
    title = r"""
          _____   _____ *    * _____ **  ** ______ _____  ______  _____ 
     /\   |  ** \ / **__| |  | |_   *|  \/  |  *___|  ** \|  **__|/ ____|
    /  \  | |__) | |    | |__| | | | | \  / | |__  | |  | | |__  | (___  
   / /\ \ |  *  /| |    |  *_  | | | | |\/| |  **| | |  | |  **|  \___ \ 
  / ____ \| | \ \| |____| |  | |_| |_| |  | | |____| |__| | |____ ____) |
 /_/    \_\_|  \_\\_____|_|  |_|_____|_|  |_|______|_____/|______|_____/ 
                                                                         
                                                                         
                                                        
                                   :.....:.                          
                               ::::.=:.. =:..:                       
                             -:-:=*+====-..::...                     
                             #*:==:...     .:-.:-                    
                            +-+=:==::::::::::-::=:                   
                            +*=**=:::.:.....:**--:                   
                           +-+*#*=*%%**+*%%*=*#**=*                  
                             %%***%*+*#-:#+==#=#%#*                   
                             %%%*--=:-:.::-:.-#%%                    
                              **+*=:*=:.:=::+==*                     
                               *=+++::=%%-.:=::-=                     
                               **=:-=%**+*%=.-=--                     
                                **=+*=-:::==:::+                      
                             +*=* #*-=:-=-:::+==-                      
                          ===++*##*%#=+-*%==--***+*=+-                 
                        +*==:-=#%**##*+++#**+*++*==:=-:::-             
                        ==*=::=%****##%%%%%%*+==*:-::-::..:            
                         ===*====++*****##%%*====:-.:::::.:             
                          -===-=:=----==**#*=-:::-:::=:::.              
                           +=+:==:=:::--:-=--::.-:::.-::::              
                           ===:=-:-====-::::::=-::.:::::                
                            ++==-+-:::::==:=.:=--=::::--=               
                             +*==-*-:::::=-:=+-==::::--=                
                               +===*-:-==-=*==+=::::-=                  
                    """
    print(title)

class StockSignalAnalyzer:
    def __init__(self):
        self.symbol = None
        self.data = None
        display_title()  # Display the title when initializing
        self.setup_widgets()
        
    def setup_widgets(self):
        """Setup interactive widgets"""
        self.stock_list = ['NVDA', 'AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA']
        
        self.stock_dropdown = widgets.Dropdown(
            options=self.stock_list,
            value='NVDA',
            description='Stock:',
            style={'description_width': 'initial'}
        )
        
        self.analyze_button = widgets.Button(
            description='Generate Signals',
            button_style='success',
            style={'description_width': 'initial'}
        )
        
        self.analyze_button.on_click(self.on_analyze_click)
        self.output = widgets.Output()
        
        display(widgets.VBox([
            self.stock_dropdown,
            self.analyze_button,
            self.output
        ]))

    def fetch_data(self, period='1mo', interval='1d'):
        """Fetch stock data using yfinance"""
        try:
            self.symbol = self.stock_dropdown.value
            stock = yf.Ticker(self.symbol)
            self.data = stock.history(period=period, interval=interval)
            if not self.data.empty:
                return True
            return False
        except Exception as e:
            print(f"Error fetching data: {e}")
            return False

    def calculate_rsi(self, period=14):
        """Calculate RSI"""
        delta = self.data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def calculate_macd(self, short_period=12, long_period=26, signal_period=9):
        """Calculate MACD"""
        exp1 = self.data['Close'].ewm(span=short_period, adjust=False).mean()
        exp2 = self.data['Close'].ewm(span=long_period, adjust=False).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=signal_period, adjust=False).mean()
        return macd, signal

    def get_rsi_status(self, rsi):
        """Determine RSI status"""
        if rsi >= 70:
            return "Overbought"
        elif rsi <= 30:
            return "Oversold"
        elif rsi >= 65:
            return "Approaching Overbought"
        else:
            return "Neutral"

    def get_macd_signal(self, macd, signal):
        """Determine MACD signal strength"""
        diff = macd - signal
        if diff > 2:
            return "Strong Bullish"
        elif diff > 0:
            return "Bullish"
        elif diff < -2:
            return "Strong Bearish"
        else:
            return "Bearish"

    def generate_trading_signal(self, rsi_status, macd_signal):
        """Generate trading signal based on RSI and MACD"""
        if rsi_status == "Overbought" and "Bearish" in macd_signal:
            return "SELL"
        elif rsi_status == "Oversold" and "Bullish" in macd_signal:
            return "BUY"
        elif "Strong Bullish" in macd_signal and rsi_status != "Overbought":
            return "BUY"
        elif "Strong Bearish" in macd_signal and rsi_status != "Oversold":
            return "SELL"
        else:
            return "HOLD"

    def analyze_signals(self):
        """Analyze stock signals"""
        if self.data is None or self.data.empty:
            return None
        
        # Calculate indicators
        self.data['RSI'] = self.calculate_rsi()
        macd, signal = self.calculate_macd()
        self.data['MACD'] = macd
        self.data['Signal'] = signal
        
        # Calculate volume change
        self.data['Volume_Change'] = self.data['Volume'].pct_change() * 100
        
        # Generate signals
        results = []
        for i in range(len(self.data)):
            if i > 0:  # Skip first row due to pct_change
                date = self.data.index[i]
                rsi = self.data['RSI'].iloc[i]
                rsi_status = self.get_rsi_status(rsi)
                macd_signal = self.get_macd_signal(
                    self.data['MACD'].iloc[i],
                    self.data['Signal'].iloc[i]
                )
                
                results.append({
                    'Trading_Date': date.strftime('%Y-%m-%d'),
                    'Stock_Symbol': self.symbol,
                    'Opening_Price': self.data['Open'].iloc[i],
                    'Closing_Price': self.data['Close'].iloc[i],
                    'Volume': self.data['Volume'].iloc[i],
                    'Volume_Change': self.data['Volume_Change'].iloc[i],
                    'RSI_Status': f"{rsi_status} ({rsi:.2f})",
                    'MACD_Signal': macd_signal,
                    'Trading_Signal': self.generate_trading_signal(rsi_status, macd_signal)
                })
        
        return pd.DataFrame(results)

    def on_analyze_click(self, b):
        """Handle analyze button click"""
        with self.output:
            clear_output(wait=True)
            if self.fetch_data():
                signals_df = self.analyze_signals()
                if signals_df is not None:
                    # Style the dataframe
                    styled_df = signals_df.style.apply(lambda x: [
                        'background-color: #90EE90' if signal == 'BUY'
                        else 'background-color: #FFB6C1' if signal == 'SELL'
                        else '' for signal in x
                    ], subset=['Trading_Signal'])
                    
                    display(styled_df)
                    
                    # Plot RSI and MACD
                    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12))
                    
                    # Price plot
                    ax1.plot(self.data.index, self.data['Close'])
                    ax1.set_title(f'{self.symbol} Price')
                    ax1.grid(True)
                    
                    # RSI plot
                    ax2.plot(self.data.index, self.data['RSI'])
                    ax2.axhline(y=70, color='r', linestyle='--')
                    ax2.axhline(y=30, color='g', linestyle='--')
                    ax2.set_title('RSI')
                    ax2.grid(True)
                    
                    # MACD plot
                    ax3.plot(self.data.index, self.data['MACD'], label='MACD')
                    ax3.plot(self.data.index, self.data['Signal'], label='Signal')
                    ax3.set_title('MACD')
                    ax3.grid(True)
                    ax3.legend()
                    
                    plt.tight_layout()
                    plt.show()
            else:
                print("Failed to fetch data")

def main():
    analyzer = StockSignalAnalyzer()

if __name__ == "__main__":
    main()